# 了解 Self-attention 的基本数学思路

In [3]:
import torch
from torch.nn import functional as F

## 1. 生成几批序列数据

In [5]:
# 生成一个3维张量作为输入
# batch, time, channels。批次，token位置，token向量各维度参数
B, T, C = 2, 8, 2
x = torch.rand(B, T, C)
# 查看x形状
x.shape

torch.Size([2, 8, 2])

In [6]:
# 查看x内容，把这看做两批数据(B),每批有8个token(T)，每个token用2个参数描述(C)
x

tensor([[[0.0687, 0.4276],
         [0.0396, 0.3199],
         [0.4048, 0.4472],
         [0.1638, 0.5660],
         [0.6666, 0.0691],
         [0.5223, 0.1797],
         [0.1815, 0.9351],
         [0.8868, 0.3212]],

        [[0.1570, 0.9270],
         [0.8815, 0.3526],
         [0.0064, 0.6480],
         [0.6853, 0.6159],
         [0.8965, 0.9815],
         [0.2357, 0.9653],
         [0.3571, 0.7171],
         [0.8771, 0.6371]]])

## 2. 考虑使token注意到前面的token
* 我们希望这8个token能够相互交流（注意 attention 到）
* 同时我们希望每个 token 仅注意到它之前的 token (以更像在生成)，如第5个token仅能注意到前4个token
* 最简单的让 token 获取前方信息的方式就是把前面全部取平均，得到一个向量去描述前面信息（平均通常没什么用，且丢失了前方的token的位置信息，但用于示例很方便）。未来再考虑用其他的方式获取前面信息

### 2.1 先考虑最简单粗暴的取前面平均值的方式：循环

In [9]:
# 对于张量可以用这种方式抽取其中一部分
x[0, :2]

tensor([[0.0687, 0.4276],
        [0.0396, 0.3199]])

In [10]:
# bow means bag of words，表征抽取前面的信息
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]  # (t,C)
        # 对到自己的位置的值取平均
        xbow[b, t] = torch.mean(xprev, 0)

In [11]:
# 原始输入
x[0]

tensor([[0.0687, 0.4276],
        [0.0396, 0.3199],
        [0.4048, 0.4472],
        [0.1638, 0.5660],
        [0.6666, 0.0691],
        [0.5223, 0.1797],
        [0.1815, 0.9351],
        [0.8868, 0.3212]])

In [12]:
# 对前面取平均后的结果
xbow[0]

tensor([[0.0687, 0.4276],
        [0.0541, 0.3738],
        [0.1710, 0.3982],
        [0.1692, 0.4402],
        [0.2687, 0.3660],
        [0.3110, 0.3349],
        [0.2925, 0.4207],
        [0.3667, 0.4082]])

### 2.2 换用矩阵乘法取平均的方式
循环的方式比较直观，但低效，且不方便推广。 
而矩阵是更方便的

In [14]:
# 由于只对前面取平均，可以用 下三角矩阵。0位置自然就会在乘法后被忽略掉
a = torch.tril(torch.ones(4, 4))
a

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [15]:
# 用这样的矩阵乘一个列向量等效在对列向量进行加权求和了（每行为一系列权重）。但由于我们是在取平均值，所以需要让每一行的和归一化
# 先按行求和
a_sum_by_1 = torch.sum(a, 1, keepdim=True)
a_sum_by_1

tensor([[1.],
        [2.],
        [3.],
        [4.]])

In [16]:
# 按行除
a = a / a_sum_by_1
a

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])

In [17]:
# 基于上面的思路再求前面平均值的 xbow2
# 先得到权重矩阵
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [18]:
# 再用这个矩阵乘 x 得到 xbow2
# (B,T,T) @ (B,T,C) ---> (B,T,C)
xbow2 = wei @ x
xbow2

tensor([[[0.0687, 0.4276],
         [0.0541, 0.3738],
         [0.1710, 0.3982],
         [0.1692, 0.4402],
         [0.2687, 0.3660],
         [0.3110, 0.3349],
         [0.2925, 0.4207],
         [0.3667, 0.4082]],

        [[0.1570, 0.9270],
         [0.5193, 0.6398],
         [0.3483, 0.6425],
         [0.4326, 0.6359],
         [0.5254, 0.7050],
         [0.4771, 0.7484],
         [0.4599, 0.7439],
         [0.5121, 0.7306]]])

In [19]:
# 对比两种方式计算的xbow，应该是一样的
torch.allclose(xbow, xbow2)

True

### 2.3 进一步我们利用 softmax 函数
softmax 可以把一堆实数映射到 [0,1] 区间，并保证映射后的和为1，非常适合用于计算概率

In [21]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
# softmax 会把 -inf 映射为0，所以需要把希望为0的地方置为 -inf
wei = wei.masked_fill(tril == 0, float("-inf"))
# 对各个最后一维，即行，进行softmax
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

可见可以得到和前面用sum除一样的结果

In [23]:
# 同样方式计算xbow3
xbow3 = wei @ x
xbow3

tensor([[[0.0687, 0.4276],
         [0.0541, 0.3738],
         [0.1710, 0.3982],
         [0.1692, 0.4402],
         [0.2687, 0.3660],
         [0.3110, 0.3349],
         [0.2925, 0.4207],
         [0.3667, 0.4082]],

        [[0.1570, 0.9270],
         [0.5193, 0.6398],
         [0.3483, 0.6425],
         [0.4326, 0.6359],
         [0.5254, 0.7050],
         [0.4771, 0.7484],
         [0.4599, 0.7439],
         [0.5121, 0.7306]]])

In [24]:
# xbow3 也能和前两种方式结果一致
torch.allclose(xbow, xbow3)

True

## 3. 用self-attention的方式

In [26]:
# 扩展一下输入数据的大小，重新设置输入数据
B, T, C = 4, 8, 32
x = torch.rand(B, T, C)
x.shape

torch.Size([4, 8, 32])

### 3.1 实现不带v的单头注意力机制

In [28]:
import torch.nn as nn
from torch.nn import functional as F

# 注意头的大小设置
head_size = 16
# 每个头需要一个 key 矩阵，作用到x上以后提取x的特征信息
key = nn.Linear(C, head_size, bias=False)
# 每个头需要一个 query 矩阵，作用到x上以后提取想问的问题
query = nn.Linear(C, head_size, bias=False)
# 作用于x得到具体的key和query
k = key(x)  # (B,T,16)
q = query(x)  # (B,T,16)

In [29]:
k.shape

torch.Size([4, 8, 16])

In [30]:
q.shape

torch.Size([4, 8, 16])

In [31]:
# 让 k和q进行内积，获得key和query到底有多么匹配
# 为了求内积，需要转置一下后两维
# (B,T,16) @ (B,16,T) --> (B,T,T)
# 注：忽略B以后，k 和 q 都是由T个行向量组成的矩阵，内积有T*T组，对应T*T结果
# 这个 wei 类似之前的取平均的注意力矩阵类似，表征每个token应该有多么关心其他各个token
wei = q @ k.transpose(-2, -1)
wei.shape

torch.Size([4, 8, 8])

In [32]:
# 查看第一个batch的wei的权重矩阵
wei[0]

tensor([[-0.5484,  0.0883,  0.3871, -0.2779,  0.0643,  0.5207,  0.2659, -0.4657],
        [-0.0583,  0.1873,  0.5663,  0.3572,  0.0514,  0.7701,  0.9311, -0.1547],
        [-0.8897, -0.4039,  0.0098, -0.5331, -0.2614,  0.1063, -0.1382, -0.8162],
        [-0.7110, -0.3480,  0.0835, -0.5274, -0.6238,  0.0388,  0.0916, -0.9112],
        [-0.5468, -0.1322,  0.3889, -0.2240, -0.1973,  0.3237,  0.3277, -0.5813],
        [-0.5489, -0.2345,  0.2170, -0.3353, -0.2469,  0.0510,  0.1611, -0.5779],
        [-0.7004, -0.4023,  0.1859, -0.4054, -0.4893,  0.3774,  0.3124, -0.7932],
        [-0.8228, -0.2948, -0.1135, -0.5053, -0.4677,  0.0635, -0.0205, -0.7400]],
       grad_fn=<SelectBackward0>)

In [33]:
# 类似之前2.3的做法，我们让各个token仅关注其前面的token，并且用softmax归一化
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4389, 0.5611, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1967, 0.3197, 0.4836, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1709, 0.2456, 0.3782, 0.2053, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1272, 0.1925, 0.3242, 0.1757, 0.1804, 0.0000, 0.0000, 0.0000],
        [0.1119, 0.1533, 0.2408, 0.1386, 0.1514, 0.2039, 0.0000, 0.0000],
        [0.0767, 0.1033, 0.1860, 0.1030, 0.0947, 0.2253, 0.2111, 0.0000],
        [0.0753, 0.1278, 0.1531, 0.1035, 0.1075, 0.1828, 0.1681, 0.0819]],
       grad_fn=<SelectBackward0>)

In [34]:
# 计算加权平均
out = wei @ x
out.shape

torch.Size([4, 8, 32])

### 3.2 实现带v的单头注意力机制
刚才的做法下，仅基于key 和 query 的内积  
self-attention 还进一步对 x 进行了一次变换，用另一个矩阵v  

合在一起的思路是：
* query 是我想问我自己的一批问题
* key 是我基于我自己的信息的一批特征
* query @ key 得到我想关注我的哪些地方（权重表示）
* value 表示我各个位置想提供什么样的信息

整合输出的注意信息就是 out = (query(x) @ key(x).T)*value(x)

In [36]:
# 注意头的大小设置
head_size = 16
# 每个头需要一个 key 矩阵，作用到x上以后提取x的特征信息
key = nn.Linear(C, head_size, bias=False)
# 每个头需要一个 query 矩阵，作用到x上以后提取想问的问题
query = nn.Linear(C, head_size, bias=False)
# key,query作用于x得到具体的key和query
k = key(x)  # (B,T,16)
q = query(x)  # (B,T,16)
wei = q @ k.transpose(-2, -1)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
# 额外再加一个value矩阵直接对x进行作用
value = nn.Linear(C, head_size, bias=False)
# v 也直接作用于x
v = value(x)  # (B,T,16)
out = wei @ v

In [37]:
out.shape

torch.Size([4, 8, 16])

### 3.3 添加系数控制方差
在 attention is all your need 的论文中，计算out的时候还会除以 head_size^0.5 这么个系数  
原因是为了控制矩阵的方差不因计算而膨胀  
举例子：

In [39]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)

In [40]:
# torch.randn生成的都是均值0方差1的随机数，方差期望为1
k.var()

tensor(0.9847)

In [41]:
# torch.randn生成的都是均值0方差1的随机数，方差期望为1
q.var()

tensor(1.1053)

In [42]:
# 但计算后的wei的方差的期望却膨胀到了 head_size 倍
wei.var()

tensor(17.2691)

方差变大的原因如下：
* 如果 X,Y随机变量都满足 均值0方差1，可计算 X*Y 也满足均值0方差1  
* 由于计算 wei 即k和q的内积的时候相当于是计算了 head_size 组不同 X*Y 的和。  
* 求和会导致方差为各随机变量的方差相加，于是方差的期望变为 head_size


方差变大会有什么后果：
* 而方差增加会导致每一层的权重在方差上不一致
* 特别当方差非常大时，对应到softmax函数（scale敏感），会产生尖锐集中的分布
* 这容易导致梯度消失，进而导致训练效果受限
* 同时我们也不希望每个 token 仅向少量几个其他 token 获取信息（wei的分布不应太尖锐和集中）
* 所以需要添加一个系数去控制方差

举个例子看softmax的尺度敏感性

In [44]:
# 生成一个一维张量
a = torch.randn(T)
a

tensor([-0.4427,  0.8926, -0.7062, -0.7538,  1.3745, -0.1691, -1.4154, -0.6329])

In [45]:
# 取softmax
b = torch.softmax(a, dim=-1)
b

tensor([0.0668, 0.2538, 0.0513, 0.0489, 0.4110, 0.0878, 0.0252, 0.0552])

In [46]:
# 把所有元素扩大10倍以后取softmax
# 可以看到往a里面最大地那个值，占有了绝大部分b的结果，即集中度更高了
# 这是由于softmax是指数函数作用后的归一，而指数函数是 scale 敏感的
b = torch.softmax(a * 10, dim=-1)
b

tensor([1.2716e-08, 8.0136e-03, 9.1273e-10, 5.6675e-10, 9.9199e-01, 1.9629e-07,
        7.5872e-13, 1.8981e-09])

由于计算已知方差扩大了 head_size 倍，所以仅需对结果除以 head_size^0.5 则能达到控制方差的目标。  
所以修改后的 attention 代码如下

In [48]:
# 注意头的大小设置
head_size = 16
# 每个头需要一个 key 矩阵，作用到x上以后提取x的特征信息
key = nn.Linear(C, head_size, bias=False)
# 每个头需要一个 query 矩阵，作用到x上以后提取想问的问题
query = nn.Linear(C, head_size, bias=False)
# key,query作用于x得到具体的key和query
k = key(x)  # (B,T,16)
q = query(x)  # (B,T,16)
wei = q @ k.transpose(-2, -1)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
# 额外再加一个value矩阵直接对x进行作用
value = nn.Linear(C, head_size, bias=False)
# v 也直接作用于x
v = value(x)  # (B,T,16)
# 计算注意后的结果，并且乘以系数稳定方差的期望
out = wei @ v * head_size**-0.5